# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cantidad de oraciones: 54607542. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [5]:
import sys
sys.path.append('../')
from corpus_WiNER.corpus_utils import *

import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import gensim
import os
import random
import itertools
from gensim.models import Word2Vec, KeyedVectors

### Cargamos los datos

In [2]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

/users/ekokic/thesis_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(word_mapping.shape)
word_mapping.head()

(2021984, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [4]:
len(word_mapping['word'].unique())

2021983

### Cargamos los artículos del Documento "0"

In [5]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [6]:
doc_0.shape

(130209, 2)

In [7]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [8]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [9]:
doc_0 = doc_0.dropna()

In [10]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [11]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [12]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [13]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [14]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [15]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [16]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [2]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 10.941645860671997


In [18]:
print('Cantidad de word embeddings: {}'.format(len(w2v_model.vectors)))

Cantidad de word embeddings: 3000000


In [19]:
print('Dimensionalidad de los vectores: {}'.format(w2v_model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google se tiene $D$ = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$
   

<br>
Al tomar una ventana simétrica, se realiza un relleno (padding) con ceros a izquierda y/o derecha según corresponda para mantener la misma dimesionalidad en cada nuevo vector generado.

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [20]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [21]:
print(coarseNE_0.shape)
coarseNE_0.head()

(259470, 2)


,named-entity,art_ID
0,NaN,1000.0
1,0\t0\t2\t0,NaN
2,0\t5\t6\t1,NaN
3,0\t10\t12\t0,NaN
4,1\t2\t4\t0,NaN


Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [22]:
coarseNE_0 = spread_artID(coarseNE_0)
coarseNE_0 = coarseNE_0.dropna()

In [23]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

coarseNE_0 contiene 257656 entidades.
coarseNE_0 contiene 1810 artículos


Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [24]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [25]:
coarseNE_0.head()

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER
4,1000.0,1,2,4,PER
5,1000.0,1,5,6,PER


No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

Notar que esto sucede porque esos artículos no contienen entidades nombradas.

In [26]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

Artículos que están presentes en Documents/0 pero no en CoarseNE/0: [431177.0, 432375.0, 432318.0, 10263.0]


In [27]:
article = doc_0[doc_0.art_ID == 431177]
for sentence in article.sentence.values:
    dec_sentence = sentence_decoder(sentence, word_mapping)
    print(' '.join(dec_sentence))

In economics , economics of location is a strategy used by firms in a monopolistic competition environment .
Unlike a product differentiation strategy , where firms make their products different in order to attract customers , the economics of location strategy causes firms to produce similar or identical products .


### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [28]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

In [29]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]

In [30]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Hercule Poirot : PER
Belgian : LOC
Agatha Christie : PER


# Ahora utilicemos los dataframes preprocesados

In [31]:
doc77_df = pd.read_pickle('./corpus_WiNER/docs_df/77')
article_df = doc77_df[doc77_df.art_ID == 145492]
dec_sentence = article_df.iloc[0, article_df.columns.get_loc('sentence')]
doc77_df.head()

,sentence,art_ID
1,"[Belton, House, is, a, Grade, I, listed, count...",145492.0
2,"[The, mansion, is, surrounded, by, formal, gar...",145492.0
3,"[Belton, has, been, described, as, a, compilat...",145492.0
4,"[The, house, has, also, been, described, as, t...",145492.0
5,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0


In [32]:
coarseNE_77 = pd.read_pickle('./corpus_WiNER/coarseNE_df/77')
art_entities_df = coarseNE_77[coarseNE_77.art_ID == 145492]
sen_entities_0 = art_entities_df[art_entities_df.senIdx == 0]
art_entities_df.head()

,art_ID,senIdx,begin,end,entityType
42515,145492.0,0,0,2,LOC
42516,145492.0,0,10,11,LOC
42517,145492.0,0,12,13,LOC
42518,145492.0,0,14,15,LOC
42519,145492.0,0,16,17,LOC


In [33]:
for idx, row in sen_entities_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Belton House : LOC
Belton : LOC
Grantham : LOC
Lincolnshire : LOC
England : LOC


In [34]:
entity_list = entityListFromSentence(senIdx=0, sen_length=len(dec_sentence),
                                     art_entities_df=art_entities_df)
print(' '.join(article_df['sentence'][1]))
print(entity_list)

Belton House is a Grade I listed country house in Belton near Grantham , Lincolnshire , England .
['LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O']


In [35]:
start = time.time()
wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                   strategy='exp_decay',
                                                   W=5, w2v_model=w2v_model)
end = time.time()
print('demora:', end-start)

demora: 3.4760005474090576


In [36]:
entity_list = list(wordVector_Entity_df['entityType'])
tokens = len(entity_list)
print('Cantidad de palabras: {}'.format(tokens))
n_e = len(entity_list) - entity_list.count('O')
print('Cantidad de entidades nombradas: {}'.format(n_e))
print('Porcentaje de entidades nombradas: {:.2f}%'.format(n_e/tokens*100))

Cantidad de palabras: 4395
Cantidad de entidades nombradas: 399
Porcentaje de entidades nombradas: 9.08%


En general, cada artículo contiene una proporción reducida de palabras que son entidades.

Una alternativa para evitar cómputo y uso de memoria podría ser eliminar una proporción de vectores que no representan ninguna entidad (etiquetados con 'O') a la hora de construir la matriz de palabra - entidad.

In [37]:
wordVector_Entity_df.head(10)

,wordVector,entityType
0,"[-0.003621502994722617, 0.012132806459215266, ...",LOC
1,"[-0.08827082592044064, -0.13598437701614993, 0...",LOC
2,"[-0.16997208299104571, 0.0033914764131283102, ...",O
3,"[-0.3915967207881669, -0.12179563739202309, 0....",O
4,"[0.048302400391625866, -0.09540661652931712, 0...",O
5,"[-0.5453598294447486, -0.15736036498200667, 0....",O
6,"[-0.12074501704763917, 0.06705739051063955, 0....",O
7,"[-0.014825387882357826, -0.25825100262032796, ...",O
8,"[-0.10727509902624327, 0.04387436488263421, 0....",O
9,"[0.07985290764610813, -0.14920335335911, -0.03...",O


In [38]:
drop_non_entities(wordVector_Entity_df.head(10), 0.50)

,wordVector,entityType
0,"[-0.003621502994722617, 0.012132806459215266, ...",LOC
1,"[-0.08827082592044064, -0.13598437701614993, 0...",LOC
5,"[-0.5453598294447486, -0.15736036498200667, 0....",O
6,"[-0.12074501704763917, 0.06705739051063955, 0....",O
8,"[-0.10727509902624327, 0.04387436488263421, 0....",O
9,"[0.07985290764610813, -0.14920335335911, -0.03...",O


# División de los documentos

### Cargamos los primeros 2000 documentos que van a ser utilizados para entrenamiento

In [3]:
doc_filenames, coarseNE_filenames = read_filenames()
doc_filenames = doc_filenames[0:2000]
coarseNE_filenames = coarseNE_filenames[0:2000]
docs_df, coarseNE_df = load_docs(doc_filenames, coarseNE_filenames)

In [4]:
print('Cantidad de oraciones:', docs_df.shape[0])

Cantidad de oraciones: 40865743


In [5]:
print('Cantidad de artículos que no contienen ninguna entidad:', 
      len(docs_df.art_ID.unique()) - len(coarseNE_df.art_ID.unique()))

Cantidad de artículos que no contienen ninguna entidad: 5376


Nos quedamos con aquellos artículos que contienen al menos una entidad nombrada.

In [6]:
# art_IDs = coarseNE_df.art_ID.unique() # Solo se ejecuta la primera vez sino no fueron guardados

#### Extraemos una muestra aleatoria de 1000 artículos

In [7]:
# np.random.seed(42)
# np.random.shuffle(art_IDs)
# art_IDs_sample = art_IDs[0:1000]

#### Guardamos los IDs de los articulos seleccionados en la muestra

In [8]:
# art_IDs_sample_df = pd.DataFrame(data=art_IDs_sample, columns=['art_ID'])
# art_IDs_sample_df.to_csv('./corpus_WiNER/art_IDs_sample_train.csv', index=False)

#### Cargamos

In [9]:
art_IDs_sample_train = pd.read_csv('corpus_WiNER/art_IDs_sample_train.csv')
art_IDs_sample_train = np.array(art_IDs_sample_train['art_ID'])

#### Filtramos

In [10]:
articles_df = docs_df[docs_df.art_ID.isin(art_IDs_sample)]
entities_df = coarseNE_df[coarseNE_df.art_ID.isin(art_IDs_sample)]

In [11]:
articles_df.shape

(20870, 2)

In [16]:
start = time.time()
genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='exp_decay', W=5, w2v_model=w2v_model, splitType='train')
end = time.time()
print('Demora total: {}'.format(end - start))

10it [00:18,  1.88s/it]              /users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1000it [06:16,  2.66it/s]


tiempo de guardado: 20.05007243156433
Demora total: 396.6113295555115


### Generación de instancias utilizando una ventana simétrica de palabras que rodea a la palabra objetivo. 

Estas instancias van a ser utilizadas en los modelos: CNN y ladder CNN.

In [13]:
art_IDs_sample_train = pd.read_csv('corpus_WiNER/art_IDs_sample_train.csv')
art_IDs_sample_train = np.array(art_IDs_sample_train['art_ID'])

In [14]:
generate_cnn_instances(art_IDs_sample_train, articles_df, entities_df, W=2, splitType='train')

1000it [00:31, 31.95it/s]            
20870it [00:04, 4942.15it/s]


### Cargamos los siguientes 600 documentos que se van a utilizar para VALIDACIÓN

In [3]:
doc_filenames, coarseNE_filenames = read_filenames()
doc_filenames = doc_filenames[2000:2600]
coarseNE_filenames = coarseNE_filenames[2000:2600]
docs_df, coarseNE_df = load_docs(doc_filenames, coarseNE_filenames)

Nos quedamos con aquellos artículos que contienen al menos una entidad nombrada.

In [4]:
# art_IDs = coarseNE_df.art_ID.unique()

#### Extraemos una muestra aleatoria de 250 artículos

In [5]:
# np.random.seed(42)
# np.random.shuffle(art_IDs)
# art_IDs_sample = art_IDs[0:250]
#### Guardamos los IDs de los articulos seleccionados en la muestra
# art_IDs_sample_df = pd.DataFrame(data=art_IDs_sample, columns=['art_ID'])
# art_IDs_sample_df.to_csv('./corpus_WiNER/art_IDs_sample_dev.csv', index=False)
#### Cargamos
art_IDs_sample_dev = pd.read_csv('corpus_WiNER/art_IDs_sample_dev.csv')
art_IDs_sample_dev = np.array(art_IDs_sample_dev['art_ID'])
#### Filtramos
articles_df = docs_df[docs_df.art_ID.isin(art_IDs_sample)]
entities_df = coarseNE_df[coarseNE_df.art_ID.isin(art_IDs_sample)]

In [8]:
start = time.time()
genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='exp_decay', W=2, w2v_model=w2v_model, splitType='dev')
end = time.time()
print('Demora total: {}'.format(end - start))

88it [00:08, 10.10it/s]              /users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
250it [00:22, 11.06it/s]


tiempo de guardado: 3.744479179382324
Demora total: 26.364149570465088


In [9]:
## Generación de instancias utilizando una ventana simétrica de palabras que rodea a la palabra objetivo. 
# Estas instancias van a ser utilizadas en los modelos: CNN y ladder CNN.
art_IDs_sample_dev = pd.read_csv('corpus_WiNER/art_IDs_sample_dev.csv')
art_IDs_sample_dev = np.array(art_IDs_sample_dev['art_ID'])
generate_cnn_instances(art_IDs_sample_dev, articles_df, entities_df, W=2, splitType='dev')

250it [00:04, 61.33it/s]             
2739it [00:00, 5869.68it/s]


### Cargamos los siguientes 600 documentos que se van a utilizar para TEST

In [10]:
doc_filenames, coarseNE_filenames = read_filenames()
doc_filenames = doc_filenames[2600:3200]
coarseNE_filenames = coarseNE_filenames[2600:3200]
docs_df, coarseNE_df = load_docs(doc_filenames, coarseNE_filenames)

Nos quedamos con aquellos artículos que contienen al menos una entidad nombrada.

In [11]:
# art_IDs = coarseNE_df.art_ID.unique()

#### Extraemos una muestra aleatoria de 250 artículos

In [12]:
# np.random.seed(42)
# np.random.shuffle(art_IDs)
# art_IDs_sample = art_IDs[0:250]
#### Guardamos los IDs de los articulos seleccionados en la muestra
# art_IDs_sample_df = pd.DataFrame(data=art_IDs_sample, columns=['art_ID'])
# art_IDs_sample_df.to_csv('./corpus_WiNER/art_IDs_sample_test.csv', index=False)
#### Cargamos
art_IDs_sample_test = pd.read_csv('corpus_WiNER/art_IDs_sample_test.csv')
art_IDs_sample_test = np.array(art_IDs_sample_test['art_ID'])
#### Filtramos
articles_df = docs_df[docs_df.art_ID.isin(art_IDs_sample)]
entities_df = coarseNE_df[coarseNE_df.art_ID.isin(art_IDs_sample)]

In [15]:
start = time.time()
genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='exp_decay', W=2, w2v_model=w2v_model, splitType='test')
end = time.time()
print('Demora total: {}'.format(end - start))

233it [00:26,  8.89it/s]             /users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
250it [00:28,  8.87it/s]


tiempo de guardado: 5.263600587844849
Demora total: 33.460793256759644


In [17]:
## Generación de instancias utilizando una ventana simétrica de palabras que rodea a la palabra objetivo. 
# Estas instancias van a ser utilizadas en los modelos: CNN y ladder CNN.
art_IDs_sample_test = pd.read_csv('corpus_WiNER/art_IDs_sample_test.csv')
art_IDs_sample_test = np.array(art_IDs_sample_test['art_ID'])
generate_cnn_instances(art_IDs_sample_test, articles_df, entities_df, W=2, splitType='test')

250it [00:05, 48.50it/s]             
3243it [00:00, 5026.23it/s]
